In [ ]:
pip install pybit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 2.8 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 112 kB 9.1 MB/s 
  Created wheel for pybit: filename=pybit-2.3.1-py2.py3-none-any.whl size=47628 sha256=eec4146924b49586e12fdf8479f50f84cd6f885cae02a2cb60fe4981398a3b31
  Stored in directory: /root/.cache/pip/wheels/79/0c/e6/8b0ad58167b8f53a60e8d11133b8a0b046be56cf7b55c193d9
Successfully built pybit


In [ ]:
pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ta: filename=ta-0.10.1-py3-none-any.whl size=28985 sha256=c9cd81df9f486a6dfb795a7141edb03f847169dac734d29fb5eb6475f1f93c58
  Stored in directory: /root/.cache/pip/wheels/bc/2a/c2/a56e77d07edc16a1fa7fb012667e55cb0643cfa65996bddecc
Successfully built ta


In [ ]:
from pybit import spot
import pandas as pd
import ta

In [ ]:
api_key = '1111111'
api_secret = '1111111111111111'

In [ ]:
session = spot.HTTP(
    endpoint="https://api.bybit.com",
    api_key=api_key,
    api_secret=api_secret
)

In [ ]:
def klines(symbol):
  kline = session.query_kline(symbol=symbol, interval="1m")
  kline = kline['result']
  df = pd.DataFrame(kline)
  df = df.iloc[:,:6]
  df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
  df = df.set_index('Time')
  df.index = pd.to_datetime(df.index, unit='ms')
  df = df.astype(float)
  return df

def strategy(symbol, qty, open_position = False):

  while True:
    df = klines(symbol)
    if not open_position:
      if ta.trend.macd_diff(df.Close).iloc[-1] > 0 \
      and ta.trend.macd_diff(df.Close).iloc[-2] < 0:
        order = session.place_active_order(symbol=symbol, side="Buy", type="MARKET", qty=qty, timeInForce="GTC")
        print(order)
        open_position = True
        break

    if open_position:
      while True:
        df = klines(symbol)
        if ta.trend.macd_diff(df.Close).iloc[-1] < 0 \
        and ta.trend.macd_diff(df.Close).iloc[-2] > 0:
          order = session.place_active_order(symbol=symbol, side="Sell", type="MARKET", qty=qty, timeInForce="GTC")
          print(order)
          open_position = False
          break

In [ ]:
strategy('WLKNUSDT', 1)

{'ret_code': 0, 'ret_msg': '', 'ext_code': None, 'ext_info': None, 'result': {'accountId': '6043046', 'symbol': 'WLKNUSDT', 'symbolName': 'WLKNUSDT', 'orderLinkId': '1658781423867560', 'orderId': '1208340595058193408', 'transactTime': '1658781423872', 'price': '0', 'origQty': '1', 'executedQty': '0', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY'}}
